In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
from tensorflow import keras
if len(physical_devices) > 0:
    print(physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [4]:
nSamples = 200
trainSamples = int( 0.7*nSamples)
testSamples = int(0.1*nSamples)
valSamples = int(0.2*nSamples)
refinement = 0.4
lenGrid = 200

def getLoadData():

    allLoads = np.empty((nSamples, 3))

    load = pd.read_csv('Data_CSV/load.csv', delimiter="\t")

    print(load.columns)

    sideLoad = load[' Side Load '].to_numpy()
    upperLoad = load[' Upper Load '].to_numpy()
    lowerLoad = load[' Lower Load '].to_numpy()

    for i in range(0, nSamples):
        allLoads[i][0] = sideLoad[i]
        allLoads[i][1] = upperLoad[i]
        allLoads[i][2] = lowerLoad[i]

    return allLoads


def SplitData ():

    maskedIndexTotal_shfld =[]


    maskedIndexFile = open('maskedIndexTotal.pkl', 'rb')

    inputData = np.load('inputData.npy')
    outputData = np.load('outputData.npy')
    maskedIndexTotal = pickle.load(maskedIndexFile)


    maskedIndexFile.close()

    allLoads = getLoadData()

    # maskedIndexTotal = np.array(maskedIndexTotal)

    print(inputData.shape, outputData.shape, allLoads.shape)

    # for i in range(0, nSamples):
    #     print(str(len(maskedIndexTotal[i][0]))+"\t"+str(len(maskedIndexTotal[i][0]))+"\n")

    randIndices = np.arange(0, nSamples)
    np.random.shuffle(randIndices)

    inputData = inputData[randIndices][:][:]
    outputData = outputData[randIndices][:][:]
    allLoads = allLoads[randIndices][:]

    for i in randIndices:
        maskedIndexTotal_shfld.append(maskedIndexTotal[i])

  

    trainInput1 = inputData[0:trainSamples][:][:]
    trainOutput = outputData[0:trainSamples][:][:]

    valInput1 = inputData[trainSamples:valSamples+trainSamples][:][:]
    valOutput = outputData[trainSamples:valSamples+trainSamples][:][:]

    testInput1 = inputData[valSamples+trainSamples: valSamples+trainSamples+testSamples][:][:]
    testOutput = outputData[valSamples+trainSamples: valSamples+trainSamples+testSamples][:][:]


    trainInput2 = allLoads[0:trainSamples][:][:]
    valInput2 = allLoads[trainSamples:valSamples + trainSamples][:][:]
    testInput2 = allLoads[valSamples + trainSamples: valSamples + trainSamples + testSamples][:][:]


    return trainInput1,trainInput2,trainOutput, valInput1, valInput2, valOutput, testInput1,testInput2, testOutput

In [5]:
def modelCNN():
    input1 = keras.Input(shape=(500,500,1), batch_size=1)
    input2 = keras.Input(shape=(3,1))

    encoded = keras.layers.Conv2D(filters=150, kernel_size=(10, 10), strides=2, activation='relu')(input1)
    encoded = keras.layers.Conv2D(filters=100, kernel_size=(6, 6), strides=2, activation='relu')(encoded)
    encoded = keras.layers.Conv2D(filters=50, kernel_size=(3, 3), strides=2, activation='relu')(encoded)
    encoded = keras.layers.Conv2D(filters=25, kernel_size=(2, 2), strides=2, activation='relu')(encoded)


    flatten = keras.layers.Reshape(target_shape=(22500,1))(encoded)
    merged = keras.layers.Concatenate(axis=1)([flatten, input2])

    dense = keras.layers.Flatten()(merged)

    dense1 = keras.layers.Dense(22500)(dense)

    reshape = keras.layers.Reshape(target_shape=(30, 30, 25))(dense1)

    decoded = keras.layers.Conv2DTranspose(filters=25, kernel_size=(2, 2), strides=2, activation='relu')(reshape)
    decoded = keras.layers.Conv2DTranspose(filters=50, kernel_size=(3, 3), strides=2, activation='relu')(decoded)
    decoded = keras.layers.Conv2DTranspose(filters=100, kernel_size=(6, 6), strides=2, activation='relu')(decoded)
    decoded = keras.layers.Conv2DTranspose(filters=150, kernel_size=(10, 10), strides=2, activation='relu')(decoded)
    output = keras.layers.Conv2DTranspose(filters=1, kernel_size=(1, 1),strides=1)(decoded)


    model = keras.Model(inputs=[input1, input2], outputs=output)
    model.summary()
    tf.keras.utils.plot_model(model, 'StressCNNmodel.png', show_shapes=True)
    opt = tf.keras.optimizers.Adam(lr=0.01)

    model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model


In [6]:
trainInput1,trainInput2,trainOutput, valInput1, valInput2, valOutput, testInput1,testInput2, testOutput = SplitData()

model = modelCNN()

Index(['Job No.', ' Side Load ', ' Upper Load ', ' Lower Load '], dtype='object')
(200, 500, 500) (200, 500, 500) (200, 3)


ResourceExhaustedError: OOM when allocating tensor with shape[22503,22500] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]